In [4]:
import torch
import torch.nn as nn
from onsets_and_frames import *
from torch.utils.data import DataLoader, Dataset, TensorDataset

In [11]:
dataset_A = TensorDataset(torch.tensor([0,1]))
dataset_B = TensorDataset(torch.tensor([91,92, 93, 94, 95]))

In [14]:
loader_A = DataLoader(dataset_A, shuffle=False)
loader_B = DataLoader(dataset_B, shuffle=True)

In [16]:
for e in range(10):
    for i in zip(loader_A, loader_B):
        print(i)

([tensor([0])], [tensor([91])])
([tensor([1])], [tensor([92])])
([tensor([0])], [tensor([94])])
([tensor([1])], [tensor([95])])
([tensor([0])], [tensor([95])])
([tensor([1])], [tensor([93])])
([tensor([0])], [tensor([91])])
([tensor([1])], [tensor([93])])
([tensor([0])], [tensor([92])])
([tensor([1])], [tensor([94])])
([tensor([0])], [tensor([93])])
([tensor([1])], [tensor([94])])
([tensor([0])], [tensor([93])])
([tensor([1])], [tensor([91])])
([tensor([0])], [tensor([95])])
([tensor([1])], [tensor([94])])
([tensor([0])], [tensor([93])])
([tensor([1])], [tensor([94])])
([tensor([0])], [tensor([94])])
([tensor([1])], [tensor([93])])


In [2]:
# logdir = f'runs_AE/CQTbatch32_WNet_narrow_z_dsksize{ds_ksize}_dsstride_{ds_stride}_' + '-' + datetime.now().strftime('%y%m%d-%H%M%S')
device = f'cuda' if torch.cuda.is_available() else 'cpu'
iterations = 500000
resume_iteration = None
checkpoint_interval = 1000
train_on = 'MAPS'

batch_size = 32
sequence_length = 327680
model_complexity = 48
if torch.cuda.is_available() and torch.cuda.get_device_properties(torch.cuda.current_device()).total_memory < 10e9:
    batch_size //= 2
    sequence_length //= 2
    print(f'Reducing batch size to {batch_size} and sequence_length to {sequence_length} to save memory')

learning_rate = 0.0006
learning_rate_decay_steps = 10000
learning_rate_decay_rate = 0.98

leave_one_out = None

clip_gradient_norm = 3

validation_length = sequence_length
validation_interval = 500

refresh = False
train_groups, validation_groups = ['train'], ['validation']

In [11]:
dataset = MAPS(groups=['AkPnBcht', 'AkPnBsdf', 'AkPnCGdD', 'AkPnStgb', 'SptkBGAm', 'SptkBGCl', 'StbgTGd2'], sequence_length=sequence_length, refresh=refresh)
semi_data = MAESTRO(groups=train_groups, sequence_length=sequence_length)
validation_dataset = MAPS(groups=['ENSTDkAm', 'ENSTDkCl'], sequence_length=validation_length, refresh=refresh)

Loading group AkPnCGdD:   0%|          | 0/30 [00:00<?, ?it/s]

Loading 7 groups of MAPS at data/MAPS


Loading group train:   1%|          | 6/967 [00:00<00:17, 56.33it/s]

Loading 1 group of MAESTRO at ../../MAESTRO/maestro-v2.0.0


Loading group ENSTDkCl:   0%|          | 0/30 [00:00<?, ?it/s]

Loading 2 groups of MAPS at data/MAPS


Loading group ENSTDkCl: 100%|██████████| 30/30 [00:00<00:00, 184.29it/s]


In [13]:
loader = DataLoader(dataset, batch_size, shuffle=True, drop_last=True)
semi_loader = DataLoader(semi_data, batch_size, shuffle=True, drop_last=True)
valloader = DataLoader(validation_dataset, 4, shuffle=False, drop_last=True)

In [14]:
len(loader)

6

In [15]:
len(semi_loader)

30

In [10]:
semi_loader

In [26]:
a = [0,1,2]
b = ['a','b']

In [28]:
for i in zip(a,cycle(b)):
    print(i)

(0, 'a')
(1, 'b')
(2, 'a')


In [52]:
semi_loss = torch.load('semi_loss.pt')

In [53]:
losses = torch.load('losses.pt')

In [47]:
losses + semi_loss

TypeError: unsupported operand type(s) for +: 'dict' and 'dict'

In [56]:
{k: losses.get(k, 0) + semi_loss.get(k, 0) for k in set(losses) | set(semi_loss)}

{'loss/transcription': tensor(0.6983, device='cuda:0', grad_fn=<AddBackward0>),
 'loss/reconstruction': tensor(0.3037, device='cuda:0', grad_fn=<AddBackward0>)}

In [54]:
losses

{'loss/reconstruction': tensor(0.1518, device='cuda:0', requires_grad=True),
 'loss/transcription': tensor(0.6983, device='cuda:0', requires_grad=True)}

In [55]:
semi_loss

{'loss/reconstruction': tensor(0.1518, device='cuda:0', requires_grad=True)}

In [57]:
sum(losses.values()) + sum(semi_loss.values())

tensor(1.0020, device='cuda:0', grad_fn=<AddBackward0>)

In [39]:
semi_loss

{'loss/reconstruction': tensor(0.1518, device='cuda:0', requires_grad=True)}

In [43]:
losses.items()

dict_items([('loss/reconstruction', tensor(0.1518, device='cuda:0', requires_grad=True)), ('loss/transcription', tensor(0.6983, device='cuda:0', requires_grad=True))])

In [29]:
counter = 0
for semi_batch, batch in zip(semi_loader, cycle(loader)):
    counter+=1
    print(counter)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
